In [ ]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

def gen():
    # my laptop blows chunks
    return torch.Generator(device='cpu').manual_seed(2147483647)


In [ ]:
raw_names = open('names.txt').read().splitlines()
names = (['.'] + list(n.lower()) + ['.'] for n in raw_names)
pairs = (a + b for n in names for a, b in zip(n, n[1:]))

counts = {}
for p in pairs:
    counts[p] = counts.get(p, 0) + 1

itos = {i+1: chr(ord('a') + i) for i in range(26)}
itos[0] = '.'
stoi = {v: k for k, v in itos.items()}

count_tensor = torch.zeros((27, 27), dtype=torch.int32)
for k, v in counts.items():
    count_tensor[stoi[k[0]], stoi[k[1]]] = v

# smoothing
probs = (count_tensor+1).float()

# normalise
probs /= probs.sum(1, keepdim=True)



In [ ]:

# cut and paste from Andrej

def plot_probs(N, itos=itos):
    plt.figure(figsize=(16,16), dpi=100)
    plt.imshow(N, cmap='Blues')
    for i in range(27):
        for j in range(27):
            chstr = itos[i] + itos[j]
            plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
            plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
    plt.axis('off');

plot_probs(count_tensor)

In [ ]:

def generate_name(generator):
    name = ''
    idx = 0
    while True:
        idx = torch.multinomial(probs[idx], num_samples=1, replacement=True, generator=generator)[0].item()
        if idx == 0: 
            break
        name += itos[idx]
    return name



In [ ]:
generator = gen()

for _ in range(10):
    print(generate_name(generator))

In [ ]:
def loss_fn(probs, target):
    target = [0] + [stoi[i] for i in target] + [0]
    return -torch.log(probs[target[:-1], target[1:]])
    

In [ ]:
sum(loss_fn(probs, name).sum() for name in raw_names) / sum(len(w)+1 for w in raw_names) 